In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_pickle('ResAss_w_PbSch_Rtngs_Clnd_df.p')

In [ ]:
df.head()

In [ ]:
# month : The month as January=1, December=12
df['Documentation_YearAge'] = 2017.0 - df['DocumentDate'].dt.year
#df['Documentation_month'] = df['DocumentDate'].dt.month

df.drop(['DocumentDate'], axis=1, inplace=True)

In [ ]:
#def documentation_season(df):
#    df['winter'] = (df['Documentation_month'].isin([1,2,12]).astype(int))
#    df['spring'] = (df['Documentation_month'].isin([3,4,5]).astype(int))
#    df['summer'] = (df['Documentation_month'].isin([6,7,8]).astype(int))
#    df['fall'] = (df['Documentation_month'].isin([9,10,11]).astype(int))
#    return df

In [ ]:
#df = documentation_season(df)

In [ ]:
#df.drop(['Documentation_month'], axis=1, inplace=True)

In [ ]:
#df['fall']

In [ ]:
#df.head()

In [ ]:
#def Doc_month_converter(df):
#    Doc_month_dummies = pd.get_dummies(df.Documentation_month, drop_first=False, prefix='DocMonth')
#    df = pd.concat([df, Doc_month_dummies], axis=1).drop('Documentation_month', axis=1)
#    return df

In [ ]:
#df = Doc_month_converter(df)

In [ ]:
df['TotalCost'] = df['SalePrice'] + df['AddnlCost']

In [ ]:
len(df) #, len(df.shape[1])

In [ ]:
def SaleWarning_conv(df):
    df.SaleWarning = df.SaleWarning.apply(lambda x: x.split())
    warningdummies = pd.get_dummies(df.SaleWarning.apply(pd.Series).stack(), prefix='SWarn', drop_first=False).sum(level=0)
    df = df.join(warningdummies, how='left').fillna(0.0)
    df = df.drop('SaleWarning', axis=1)
    return df

In [ ]:
df = SaleWarning_conv(df)

In [ ]:
def Zipcode_converter(df):
    Zipcode_dummies = pd.get_dummies(df.Zipcode, drop_first=False)
    df = pd.concat([df, Zipcode_dummies], axis=1).drop('Zipcode', axis=1)
    return df

In [ ]:
df = Zipcode_converter(df)

In [ ]:
def HeatSource_converter(df):
    HeatSource_dummies = pd.get_dummies(df.HeatSource, drop_first=False, prefix='HeatSource')
    df = pd.concat([df, HeatSource_dummies], axis=1).drop('HeatSource', axis=1)
    return df

In [ ]:
def HeatSystem_converter(df):
    HeatSystem_dummies = pd.get_dummies(df.HeatSystem, drop_first=False, prefix='HeatSystem')
    df = pd.concat([df, HeatSystem_dummies], axis=1).drop('HeatSystem', axis=1)
    return df

In [ ]:
def SqFTLiving_check(df):
    df = df[df.SqFtTotLiving > 500]
    return df

In [ ]:
def NumLivingUnits_check(df):
    df = df[df.NbrLivingUnits <= 2]
    return df

In [ ]:
df = HeatSource_converter(df)

In [ ]:
df = HeatSystem_converter(df)

In [ ]:
df = SqFTLiving_check(df)

In [ ]:
df = NumLivingUnits_check(df)

In [ ]:
df.drop(['Address', 'StreetName', 'StreetType', 'SellerName', 'BuyerName', 'DirectionSuffix', 
         'parcel_number', 'PROP_NAME', 'ES_ZONE', 'MS_ZONE', 'HS_ZONE', 'LEVY_JURIS', 'SalePrice', 
         'AddnlCost', 'ExciseTaxNbr', 'BldgGrade', 'SaleReason', 'SaleInstrument', 'FinBasementGrade'], axis=1, inplace=True)

In [ ]:
uplimit = min(df.TotalCost.mean() + df.TotalCost.std()*4, 2000000)
bottomlimit = 100000 # min(abs(df.TotalCost.mean() - df.TotalCost.std()*4), df.TotalCost.min())
print uplimit, bottomlimit

In [ ]:
df = df[(df.TotalCost > bottomlimit) & (df.TotalCost < uplimit)]

In [ ]:
df.TotalCost.max(), df.TotalCost.min(), len(df)

In [ ]:
df.TotalCost.describe()

In [ ]:
df.TotalCostRed = df.TotalCost/10**5
plt.hist(df.TotalCostRed, bins=50)
plt.xlabel("Housing prices $ X 100K")
plt.ylabel("Number of Houses");

In [ ]:
plt.hist(df.TotalCost, bins=50)
plt.xlabel("Housing prices $")
plt.ylabel("Number of Houses")

In [ ]:
plt.hist(np.log(df.TotalCost), bins=50);

In [ ]:
df.head()

In [ ]:
y = df.TotalCost
X = df.drop('TotalCost', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
FSM1 = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=0)
FSM1.fit(X_train, y_train)
y_predict1 = FSM1.predict(X_test).astype(int)
y_true = y_test.as_matrix()
FSM1.score(X_test, y_test)

In [ ]:
residuals = np.log(y_true) - np.log(y_predict1)
plt.hist(residuals, bins=50);

In [ ]:
plt.scatter(np.log(y_true), residuals, alpha=0.15)
plt.xlabel("log(y_true) 'Total Cost $ x 100K'")
plt.ylabel("log(y_true/y_predicted)");

In [ ]:
#mean_absolute_error(y_true, y_predict1)

In [ ]:
#median_absolute_error(y_true, y_predict1)

In [ ]:
y_predict1 = FSM1.predict(X_test)

In [ ]:
np.median(np.abs((y_true-y_predict1)/y_true)) * 100

In [ ]:
#np.median(np.abs(1 - y_predict1/y_true))

In [ ]:
#y.mean(), y.max(), y.min()

In [ ]:
#1 - np.mean(np.abs(y_predict1/y_true))

In [ ]:
#np.sqrt(mean_squared_error(y_true, y_predict1))

In [ ]:
# y_log = np.log(df.TotalCost)
# X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.15, random_state=42)

In [ ]:
# FSM2 = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=0)
# FSM2.fit(X_train_log, y_train_log)
# y_predict2 = FSM2.predict(X_test_log).astype(int)
# y_true_log = y_test_log.as_matrix()
# FSM2.score(X_test_log, y_test_log)

In [ ]:
# residuals2 = y_true_log - y_predict2
# plt.hist(residuals2, bins=50);

In [ ]:
# median_absolute_error(y_true_log, y_predict2), np.exp(median_absolute_error(y_true_log, y_predict2))

In [ ]:
#print sum(abs(residuals))/len(y_predict1) # --> Winner is RF with subsected price range
#print sum(abs(residuals2))/len(y_predict2)
# print sum(abs(residuals3))/len(y_predict3)
# print sum((residuals4))/len(y_predict4)
#print abs(1 - np.exp(sum(abs(residuals))/len(y_predict1)))*100

In [ ]:
importances = FSM1.feature_importances_
std = np.std([tree.feature_importances_ for tree in FSM1.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(28,8))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices],rotation='vertical')
plt.xlim([-1, X.shape[1]])
plt.show();

In [ ]:
import matplotlib
%matplotlib.style('ggplot')
importances = FSM1.feature_importances_
std = np.std([tree.feature_importances_ for tree in FSM1.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(8,8))
#plt.title("Feature importances", fontsize=15)
plt.bar(range(X.shape[1])[:10], importances[indices][:10],
       color="r", yerr=std[indices][:10], align="center")
plt.xticks(range(X.shape[1])[:10], X.columns[indices][:10],rotation='vertical', fontsize=15)
plt.yticks(fontsize=15)
plt.xlim([-1, 10])
#plt.gca().xaxis.label.set_fontsize(50)
plt.show();

In [ ]:
df.drop(['SWarn_13', 'HeatSystem_7', 'SWarn_16', 'SWarn_2','SWarn_56', 'HeatSystem_4', 
         'ACL_Y', 'HeatSystem_1', 'ACL_N', 'HP_NaN', 'AFL_N', 'HP_N', 'AFL_NaN', 'ACL_NaN', 'SWarn_8', 'SWarn_34', 
         'SWarn_11', 'SWarn_44', 'SWarn_40', 'HeatSystem_2', 'SWarn_41', 'SWarn_23', 'SWarn_60', 'SWarn_24',
         'HeatSystem_8', 'HeatSource_5', 'SqFtUnfinHalf','HeatSource_7', 'SWarn_54', 'HeatSource_0','HeatSystem_0', 
         'SWarn_39', 'SWarn_38', 'SWarn_30', 'SWarn_36', 'SWarn_7', 'SqFtUnfinFull', 'SWarn_4', 'SWarn_55', 'SWarn_17','SWarn_3',
         'HeatSource_6', 'SWarn_32','SWarn_20','SWarn_14','SWarn_37','SWarn_33','SWarn_21','HP_Y', 'SWarn_5','HeatSource_4',
         'SWarn_35','SWarn_19', 'SWarn_28', 'SWarn_92', 'SWarn_*', 'SWarn_57', 'SWarn_50','SWarn_53','AFL_Y','SWarn_43',
         'SWarn_43', 'SWarn_48', 'SWarn_6', 'SWarn_9'], axis=1, inplace=True)

In [ ]:
df.drop([98126,98107,98146,98178,98133,98104], axis=1, inplace=True)

In [ ]:
ys = df.TotalCost
Xs = df.drop('TotalCost', axis=1)

In [ ]:
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, test_size=0.15, random_state=42)

In [ ]:
FSMs = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=0)
FSMs.fit(Xs_train, ys_train)
ys_predict = FSMs.predict(Xs_test).astype(int)
ys_true = ys_test.as_matrix()
FSMs.score(Xs_test, ys_test)

In [ ]:
residualss = np.log(ys_true) - np.log(ys_predict)

In [ ]:
plt.hist(residualss, bins=50);

In [ ]:
plt.scatter(np.log(ys_true), residualss, alpha=0.15)
plt.xlabel("log(y_true/y_predicted)")
plt.ylabel("log(y_true) 'Total Cost $ x 100K'");

In [ ]:
ys_predict = FSMs.predict(Xs_test)

In [ ]:
np.median(np.abs((ys_true-ys_predict)/ys_true)) * 100

In [ ]:
1-np.median(np.abs((ys_true-ys_predict)/ys_true))

In [ ]:
importances = FSMs.feature_importances_
std = np.std([tree.feature_importances_ for tree in FSMs.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(28,8))
plt.title("Feature importances")
plt.bar(range(Xs.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(Xs.shape[1]), Xs.columns[indices],rotation='vertical')
plt.xlim([-1, Xs.shape[1]])
plt.show();

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
regressor = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=0)

In [ ]:
# TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
# FSMs = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=0)
dt_range = range(1, 11)
params = dict(max_depth=dt_range)

In [ ]:
def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """

    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = 1-np.median(np.abs((y_true-y_predict)/y_true))

    # Return the score
    return score

In [ ]:
# TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
# We initially created performance_metric using R2_score
from sklearn.metrics import make_scorer
scoring_fnc = make_scorer(performance_metric)

In [ ]:
# Create cross-validation sets from the training data
# ShuffleSplit works iteratively compared to KFOLD
# It saves computation time when your dataset grows
# X.shape[0] is the total number of elements
# n_iter is the number of re-shuffling & splitting iterations.
from sklearn.cross_validation import ShuffleSplit
cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.15, random_state = 42)

In [ ]:
grid = GridSearchCV(regressor, params, cv=cv_sets, scoring=scoring_fnc)

In [ ]:
grid = grid.fit(X, y)

In [ ]:
grid.best_estimator_

In [ ]:
GridBest = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=-1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [ ]:
GridBest.fit(X_train, y_train)
y_predict = GridBest.predict(X_test).astype(int)
y_true = y_test.as_matrix()
GridBest.score(X_test, y_test)

In [ ]:
y_predict = GridBest.predict(X_test)
np.median(np.abs((y_true-y_predict)/y_true)) * 100

In [ ]:
from sklearn.grid_search import RandomizedSearchCV

In [ ]:
cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.15, random_state = 0)
rand = RandomizedSearchCV(regressor, params, cv=cv_sets, scoring=scoring_fnc)

In [ ]:
rand = rand.fit(X, y)

In [ ]:
rand.best_estimator_

### ElasticNetCV 

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
from sklearn.datasets import make_regression

L1 = 0.1 

Standard Scalar and use pipeline


In [ ]:
regr = ElasticNetCV(l1_ratio=0.1, cv=10, n_jobs=-1, random_state=0)

In [ ]:
regr.fit(X_train, y_train)

In [ ]:
y_predict = regr.predict(X_test)

In [ ]:
y_true = y_test.as_matrix()
regr.score(X_test, y_test)

In [ ]:
y_predict = regr.predict(X_test)
np.median(np.abs((y_true-y_predict)/y_true)) * 100

In [ ]:
regr = ElasticNetCV(l1_ratio=0.5, cv=10, n_jobs=-1, random_state=0)
regr.fit(X_train, y_train)
y_predict = regr.predict(X_test)
y_true = y_test.as_matrix()
regr.score(X_test, y_test)

In [ ]:
np.median(np.abs((y_true-y_predict)/y_true)) * 100

### Partial Dependence Plots do not work anything else than gradient boosting resressor

In [ ]:
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble.partial_dependence import partial_dependence
from __future__ import print_function
print(__doc__)
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import pickle

In [ ]:
model = pickle.load(open('GradientBoostGridSearchFittedModel.pkl'))

In [ ]:
predict = model.predict(X_test)

In [ ]:
np.median(np.abs((y_true-predict)/y_true)) * 100

In [ ]:
residuals = np.log(y_true) - np.log(predict)
plt.scatter(np.log(y_true), residuals, alpha=0.15)
plt.xlabel("log(y_true) 'Total Cost $ x 100K'")
plt.ylabel("log(y_true/y_predicted)");

In [ ]:
importances = model.feature_importances_

In [ ]:
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(28,8))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(X.shape[1]), X.columns[indices],rotation='vertical')
plt.xlim([-1, X.shape[1]])
plt.show();

In [ ]:
import matplotlib
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(28,8))
plt.title("Feature importances", fontsize=24)
plt.bar(range(X.shape[1])[:10], importances[indices][:10],
       color="r", align="center")
plt.xticks(range(X.shape[1])[:10], X.columns[indices][:10],rotation='vertical', fontsize=24)
plt.yticks(fontsize=24)
plt.xlim([-1, 10])
#plt.gca().xaxis.label.set_fontsize(50)
plt.show();

In [ ]:
def new_prediction_matrix(df):
    df['Documentation_YearAge'] = 0
    return df

In [ ]:
X_pre = new_prediction_matrix(X)
X_pre.head()

In [ ]:
y_pre = model.predict(X_pre).astype(int)

In [ ]:
Matrix = X_pre
Matrix['TotalCost'] = y_pre

In [ ]:
Matrix.to_pickle('GB_Pickled_Matrix.p')

In [ ]:
model

In [ ]:
model_1 = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

In [ ]:
model_1.fit(X_train, y_train)

In [ ]:
def column_idxs(df, lst):
    idx_lst = []
    for name in lst:
        idx = df.columns.get_loc(name)
        idx_lst.append(idx)
    return idx_lst

In [ ]:
column_idxs(X_train, ['Documentation_YearAge', 'SqFtTotLiving', 'HS_Ranking', 'ES_Ranking'])

In [ ]:
features = [47, 8, 46, 44, (46, 44)]
names = list(X_train.columns.values)

fig, axs = plot_partial_dependence(model_1, X_train, features, feature_names=names, n_jobs=-1, grid_resolution=50, figsize=(14, 6))

fig.suptitle('Partial dependence of Housing Prices for Seattle City')
fig.subplots_adjust(top=0.5)  # tight_layout causes overlap with suptitle

#plt.subplots_adjust(top=0.9)  # tight_layout causes overlap with suptitle

#fig = plt.figure()

# target_feature = (46, 44)
# pdp, axes = partial_dependence(model_1, target_feature,
#                                X=X_train, grid_resolution=50)
# XX, YY = np.meshgrid(axes[0], axes[1])
# Z = pdp[0].reshape(list(map(np.size, axes))).T
# ax = Axes3D(fig)
# surf = ax.plot_surface(XX, YY, Z, rstride=1, cstride=1,
#                        cmap=plt.cm.BuPu, edgecolor='k')
# ax.set_xlabel(names[target_feature[0]])
# ax.set_ylabel(names[target_feature[1]])
# ax.set_zlabel('Partial dependence')
#  pretty init view
# ax.view_init(elev=22, azim=122)
# plt.colorbar(surf)
# plt.suptitle('Partial dependence of house value on\n'
#              'HS Rating and ES Rating')
# plt.subplots_adjust(top=0.9)
fig.tight_layout()
plt.show()
#fig.tight_layout()

In [ ]:
features = [47, 8, 46, 44, (46, 44)]
names = list(X_train.columns.values)

fig, axs = plot_partial_dependence(model_1, X_train, features, feature_names=names, n_jobs=-1, grid_resolution=50)

fig.suptitle('Partial dependence of Housing Prices for Seattle City')
fig.subplots_adjust(top=0.5)  # tight_layout causes overlap with suptitle

fig = plt.figure()

target_feature = (46, 44)
pdp, axes = partial_dependence(model_1, target_feature,
                               X=X_train, grid_resolution=50)
XX, YY = np.meshgrid(axes[0], axes[1])
Z = pdp[0].reshape(list(map(np.size, axes))).T
ax = Axes3D(fig)
surf = ax.plot_surface(XX, YY, Z, rstride=1, cstride=1,
                       cmap=plt.cm.BuPu, edgecolor='k')
ax.set_xlabel(names[target_feature[0]])
ax.set_ylabel(names[target_feature[1]])
ax.set_zlabel('\n\nPartial dependence')
# pretty init view
ax.view_init(elev=22, azim=122)
plt.colorbar(surf)
plt.suptitle('Partial dependence of house value on\n'
             'HS Rating and ES Rating')
plt.subplots_adjust(top=0.9)
fig.tight_layout()
plt.show()
#fig.tight_layout()

In [ ]:
plt.scatter(X['Documentation_YearAge'], y/100000, alpha=0.15)
plt.xlabel("Documentation Age")
plt.ylabel("Total Cost $ x 100K");

In [ ]:
plt.scatter(X['SqFtTotLiving'], y/100000, alpha=0.15)
plt.xlabel("SqFtTotLiving")
plt.ylabel("Total Cost $ x 100K");

In [ ]:
plt.scatter(2017.0-X['YrBuilt'], y/100000, alpha=0.15)
plt.xlabel("Building Age")
plt.ylabel("Total Cost $ x 100K");

In [ ]:
plt.scatter(np.log(X['LOTSQFT']), y/100000, alpha=0.15)
plt.xlabel("Logarithmic Lot SqFt")
plt.ylabel("Total Cost $ x 100K");

In [ ]:
plt.scatter(np.log(X['SqFtFinBasement']), y/100000, alpha=0.15)
plt.xlabel("Logarithmic SqFt Fin Basement")
plt.ylabel("Total Cost $ x 100K");

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR

In [ ]:
model = svm.SVR(kernel='linear', C=1)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
svm1_predict = model.predict(X_test)

In [ ]:
y_true = y_test.as_matrix()

In [ ]:
np.median(np.abs((y_true-svm1_predict)/y_true)) * 100 

In [ ]:
del(df)
del(X_train)
del(y_train)
del(X_test)
del(y_test)